# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [1]:
# My code below.
%load_ext dotenv
%dotenv 


In [2]:
import dask
dask.config.set({'dataframe.query-planning': True}) ## System required to use a new Dask DataFrame implementation with this code
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [3]:
import os
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/**/part.0.parquet"), recursive=True)
dd_px = dd.read_parquet(parquet_files)
dd_px.head()

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year
ticker,,,,,,,,,,
A,2000-01-03,56.330471,56.464592,48.193848,51.502148,43.463047,4674353,Health Care,Life Sciences Tools & Services,2000
A,2000-01-04,48.730328,49.266811,46.316166,47.567955,40.142937,4765083,Health Care,Life Sciences Tools & Services,2000
A,2000-01-05,47.389126,47.567955,43.141991,44.617310,37.652866,5758642,Health Care,Life Sciences Tools & Services,2000
A,2000-01-06,44.080830,44.349072,41.577251,42.918453,36.219189,2534434,Health Care,Life Sciences Tools & Services,2000
A,2000-01-07,42.247852,47.165592,42.203148,46.494991,39.237457,2819626,Health Care,Life Sciences Tools & Services,2000


In [4]:
print(dd_px.index.name)


ticker


For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [5]:
dd_px = dd_px.reset_index()  # Reset index
dd_px = dd_px.sort_values(by=['ticker', 'Date']) ## Sorted DataFrame by ticker and Date

In [6]:
## Add lags for variables Close and Adj_Close applying 'meta' by the System's request
dd_px['Close_lag'] = dd_px.groupby('ticker')['Close'].shift(1, meta=('Close_lag', 'f8'))
dd_px['Adj_Close_lag'] = dd_px.groupby('ticker')['Adj Close'].shift(1, meta=('Adj_Close_lag', 'f8'))

In [7]:
## Add returns based on Adjusted Close
dd_px['returns'] = (dd_px['Adj Close'] / dd_px['Adj_Close_lag']) - 1

In [8]:
##Add the day's High minus Low.

dd_px['hi_lo_range'] = dd_px['High'] - dd_px['Low']

In [9]:
## Assign the result to `dd_feat`
dd_feat = dd_px[['ticker', 'Close', 'Adj Close', 'Close_lag', 'Adj_Close_lag', 'returns', 'hi_lo_range']]

+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [ ]:
dd_px_pandas = dd_px.compute() ## Convert the Dask data frame to a pandas data frame

dd_px_pandas['returns'] = (dd_px_pandas['Adj Close'] / dd_px_pandas['Adj_Close_lag']) - 1  ## Calculate income

dd_px_pandas['rolling_mean_returns'] = dd_px_pandas['returns'].rolling(window=10).mean()  ## Add a rolling average return calculation with a window of 10 days

print(dd_px_pandas.head())  ## Return outputs

Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
+ Would it have been better to do it in Dask? Why?

(1 pt)


Comment: If data is not so big, using pandas is easier and more efficient for calculatiom moving average returns. Dask is better for a big data.

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.